In [1]:
%matplotlib inline
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None, "display.max_rows", None, "display.max_columns", None)
pd.options.mode.chained_assignment = None

In [3]:
config = [
    "1tls_2x2",
    "1tls_3x3",
    "1tls_4x4",
    "2tls_3x3x2",
    "4tls_3x3x2x2",
    "9tls_3x3x3x3",
][1]

In [4]:
dfs = [
    pd.read_csv(f'./{config}/con/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_con', con=True),
    pd.read_csv(f'./{config}/def/DuelingDoubleDQNAgent_lr0.0001.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
    pd.read_csv(f'./{config}/def/MaxPressureBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
    pd.read_csv(f'./{config}/def/SotlBaseline.csv', delimiter=',').assign(id_con=lambda df: df['id']+'_def', con=False),
]
None

In [5]:
for df in dfs:
    display(df.loc[df["done"] == True].head(2))

,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
311,312,123.704806,rlcontroller,1,0.137,0.137,"[1000.0, 200.0, 500.0, 200.0]",50,40.357603,7487.0,7523.0,29,0.807152,149.740000,150.460000,2.416667,False,True,rlcontroller_con,True
620,309,233.641993,rlcontroller,2,0.978,0.978,"[100.0, 200.0, 200.0, 600.0]",33,28.054979,2265.0,3986.0,23,0.850151,68.636364,120.787879,1.916667,False,True,rlcontroller_con,True


,Unnamed: 0,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
287,287,288,90.115140,rlcontroller,1,1.0,0.137,"[1000.0, 200.0, 500.0, 200.0]",25,17.236500,3400.0,3428.0,10,0.689460,136.000000,137.120000,0.833333,False,True,rlcontroller_def,False
600,600,313,138.392722,rlcontroller,2,1.0,0.978,"[100.0, 200.0, 200.0, 600.0]",31,25.713759,3557.0,3728.0,22,0.829476,114.741935,120.258065,1.833333,False,True,rlcontroller_def,False


,Unnamed: 0,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
244,244,245,0.0,maxpressurebaseline,1,1.0,0.137,"[1000.0, 200.0, 500.0, 200.0]",20,12.137099,200.0,200.0,4,0.606855,10.00,10.00,0.333333,False,True,maxpressurebaseline_def,False
489,489,245,0.0,maxpressurebaseline,2,1.0,0.978,"[100.0, 200.0, 200.0, 600.0]",12,7.051218,33.0,33.0,2,0.587602,2.75,2.75,0.166667,False,True,maxpressurebaseline_def,False


,Unnamed: 0,l,r,id,ep,con_p_rate,ctrl_con_p_rate,veh_n_p_hour,veh_n,sum_delay,sum_waiting_time,sum_acc_waiting_time,sum_queue_length,avg_delay,avg_waiting_time,avg_acc_waiting_time,avg_queue_length,TimeLimit.truncated,done,id_con,con
265,265,266,0.0,sotlbaseline,1,1.0,0.137,"[1000.0, 200.0, 500.0, 200.0]",139,128.723890,1388.0,19946.0,76,0.926071,9.985612,143.496403,6.333333,False,True,sotlbaseline_def,False
533,533,268,0.0,sotlbaseline,2,1.0,0.978,"[100.0, 200.0, 200.0, 600.0]",74,69.428477,644.0,14430.0,46,0.938223,8.702703,195.000000,3.833333,False,True,sotlbaseline_def,False


In [6]:
def avg_ep(df):
    return pd.merge(
        df[['id', 'ep', 'veh_n', 
            'sum_delay', 'sum_waiting_time', 'sum_acc_waiting_time', 'sum_queue_length',
            'avg_delay', 'avg_waiting_time', 'avg_acc_waiting_time', 'avg_queue_length'
           ]].groupby(['ep']).mean(),
        df[['id', 'ep', 'con_p_rate', 'ctrl_con_p_rate', 'veh_n_p_hour', 'id_con', 'con']].groupby(['ep']).apply(lambda df: df.head(1)),
        how='left', left_index=True, right_index=True
    ).reset_index(drop=True)

def make_df_ep(dfs):
    return (
        pd.concat([avg_ep(df) for df in dfs]).sort_values(['ep', 'id'])
        .reset_index(drop=True)
        .assign(ctrl_con_p_rate_round=lambda df: (df['ctrl_con_p_rate'] * 10).astype(int) / 10)
        .sort_index(axis=1)
    )

In [7]:
ep_df = make_df_ep(dfs)

In [8]:
display(ep_df.head(12))

,avg_acc_waiting_time,avg_delay,avg_queue_length,avg_waiting_time,con,con_p_rate,ctrl_con_p_rate,ctrl_con_p_rate_round,ep,id,id_con,sum_acc_waiting_time,sum_delay,sum_queue_length,sum_waiting_time,veh_n,veh_n_p_hour
0,9.055964,0.687747,0.702381,8.749424,False,1.000,0.137,0.1,1,maxpressurebaseline,maxpressurebaseline_def,223.338776,16.805420,8.428571,215.371429,24.269388,"[1000.0, 200.0, 500.0, 200.0]"
1,203.804758,0.852609,3.132212,188.822534,True,0.137,0.137,0.1,1,rlcontroller,rlcontroller_con,11968.121795,48.423799,37.586538,10841.759615,55.900641,"[1000.0, 200.0, 500.0, 200.0]"
2,57.294971,0.771721,1.430556,49.240486,False,1.000,0.137,0.1,1,rlcontroller,rlcontroller_def,1956.743056,26.457004,17.166667,1680.878472,34.000000,"[1000.0, 200.0, 500.0, 200.0]"
3,123.209393,0.907000,5.472118,12.063332,False,1.000,0.137,0.1,1,sotlbaseline,sotlbaseline_def,14353.409774,91.030964,65.665414,1277.853383,97.755639,"[1000.0, 200.0, 500.0, 200.0]"
4,6.699139,0.674553,0.427891,6.545513,False,1.000,0.978,0.9,2,maxpressurebaseline,maxpressurebaseline_def,110.444898,11.015196,5.134694,107.795918,16.183673,"[100.0, 200.0, 200.0, 600.0]"
5,76.637329,0.808844,1.408846,67.524484,True,0.978,0.978,0.9,2,rlcontroller,rlcontroller_con,2252.242718,22.825900,16.906149,1986.919094,28.000000,"[100.0, 200.0, 200.0, 600.0]"
6,72.990590,0.809140,1.396699,65.062860,False,1.000,0.978,0.9,2,rlcontroller,rlcontroller_def,2121.089457,22.694656,16.760383,1893.412141,27.859425,"[100.0, 200.0, 200.0, 600.0]"
7,176.234902,0.922989,3.822139,11.914165,False,1.000,0.978,0.9,2,sotlbaseline,sotlbaseline_def,13289.126866,64.369377,45.865672,848.585821,68.630597,"[100.0, 200.0, 200.0, 600.0]"
8,70.429820,0.871374,4.959350,19.189593,False,1.000,0.243,0.2,3,maxpressurebaseline,maxpressurebaseline_def,8146.715447,88.065915,59.512195,2042.752033,98.634146,"[1000.0, 900.0, 300.0, 600.0]"
9,36.703069,0.802784,2.510559,27.425468,True,0.243,0.243,0.2,3,rlcontroller,rlcontroller_con,2146.534247,45.497595,30.126712,1582.859589,56.301370,"[1000.0, 900.0, 300.0, 600.0]"


In [9]:
###################################################################################################################

In [10]:
ep_df.to_csv(f'./data/{config}.csv', sep = ',')